In [8]:
import boto3          # AWS SDK for Python
import pandas as pd   # Data analysis and manipulation tool
import jmespath       # Query language for JSON

In [6]:
# Definición de variables
region = "us-west-2"

In [ ]:
# Crear la sesión en AWS con el perfil especificado
# ---
# Es necesario establecer la sesión en AWS CLI usando:
# aws sso login --profile aws-sdk
# ---
session_services = boto3.Session(profile_name='aws-sdk')

In [ ]:
# Crear un cliente de EC2 con la sesión
ec2_client = session_services.client('ec2',region_name=region)

# Obtiene la información de EC2 en listas de diccionarios
response = ec2_client.describe_instances()

In [ ]:
# Definición de la consulta con los datos requeridos de las EC2.
# Se coloca cero como valor del index [0] en el caso de listas,
# para que el tipo de valor resultante no sea una lista.
# ---
# JMESPath is a query language for JSON. 
# You can extract and transform elements from a JSON document.
# --

consulta = jmespath.compile(
    """
    Reservations[].{
        OwnerId: OwnerId,
        OwnerIdNetInt: Instances[0].NetworkInterfaces[0].OwnerId,
        ReservationId: ReservationId,
        InstanceId: Instances[0].InstanceId,
        InstanceType: Instances[0].InstanceType,
        PlatformDetails: Instances[0].PlatformDetails,
        PrivateIpAddress:Instances[0].PrivateIpAddress,
        PrivateDnsName: Instances[0].PrivateDnsName,
        PublicIpAddress:Instances[0].PublicIpAddress,
        PublicDnsName: Instances[0].PublicDnsName,
        AvailabilityZone: Instances[0].Placement.AvailabilityZone,
        BlockDeviceMappings: Instances[0].BlockDeviceMappings[],
        NetworkInterfaces: Instances[0].NetworkInterfaces[],
        State: Instances[0].State.Name
    }
    """
)

In [ ]:
# Formatear los datos obtenidos de la consulta de EC2 con AWS SDK
data_ec2 = consulta.search(response)

# Convierte los datos JSON a un Data Frame
df = pd.DataFrame(data_ec2)

In [ ]:
# Despliega los datos consultados de las EC2
display(df)